Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import glob
import os
from scipy.signal import sosfilt, butter
import ruptures as rpt

Import Data

In [2]:
data_dir = os.path.expanduser("~/Documents/Senior/Fall/Clinic/Data Cleaner/tarsalis_data_clean")
os.chdir(data_dir)
filenames = glob.glob('*.csv')
dataframes = [pd.read_csv(f) for f in filenames]

Extract Changepoints

In [3]:
changepoints = []
for df in dataframes:
    x = df["pre_rect"].values
    cp = []
    labels = df["labels"].values
    for i in range(len(labels)):
        if i > 0:
            if labels[i] != labels[i - 1]:
                cp.append(i)
        if i == len(labels) - 1:
            cp.append(i)
    changepoints.append(cp)

In [37]:
%matplotlib qt
for df in dataframes[:10]:
    ts = df["post_rect"].values
    plt.figure()
    print(np.argmax(ts > 7.4))
    plt.plot(ts[np.argmax(ts > 7.4) : len(ts) - np.argmax(np.flip(ts) > 7.4)])


9677
5078
3081
1772
1318
8481
0
0
0
350


In [4]:
%matplotlib qt

Create low-pass filtered waveforms

In [5]:
sample_rate = 100 #Hz
cutoff      = 0.1 #Hz
sos_low_pass = butter(3, cutoff, btype = 'low', fs = sample_rate, output = 'sos')
for df in dataframes:
    df["filtered"] = sosfilt(sos_low_pass, df["pre_rect"].values)
    #df["filtered"] = np.convolve(df["pre_rect"].values, np.ones(sample_rate)/sample_rate, mode='same')

In [8]:
i = 3
fig, axs = plt.subplots(2, 1, sharex = True)
axs[0].plot(dataframes[i]["filtered"])
axs[0].vlines(changepoints[i], ymin=min(dataframes[i]["filtered"]), \
                            ymax=max(dataframes[i]["filtered"]), color = "black")
velocity = np.gradient(dataframes[i]["filtered"])
axs[1].plot(velocity)
axs[1].vlines(changepoints[i], ymin=min(velocity), \
                            ymax=max(velocity), color = "black")

In [7]:
i = 0
algo = rpt.Window(model = "l1").fit(np.expand_dims(dataframes[i]["filtered"].values,axis=1))
bkps = algo.predict(n_bkps = 20)
fig, ax_arr = rpt.display(dataframes[i]["filtered"], changepoints[i], bkps, figsize=(10, 6))
